# Learning

In [1]:
from hmm.hmm import HMM, sample_poisson_stimuli, learned_parameters, expectation_maximisation_hard_assignment
from hmm.types import IntArray

import numpy as np

ImportError: cannot import name 'sample_poisson_stimuli' from 'hmm.hmm' (/Users/oscarhedeby/Documents/KU/ModComp/hmm/hmm/hmm.py)

In [ ]:
gamma = 0.1
beta = 0.2
alpha = 0.01
rates = [1, 20]

# This is uppercase-gamma.
transition_matrix = np.array(
    [[1 - gamma, 0, gamma], [0, 1 - gamma, gamma], [beta / 2, beta / 2, 1 - beta]]
)

In [ ]:
hmm = HMM(transition_matrix, alpha, lambda z: sample_poisson_stimuli(z, rates), states=[0, 1, 2], rates=rates)

### Simulated data

In [ ]:
num_nodes = 8
time_steps = 100
initial_c = 2

In [ ]:
observed_processing_modes, observed_focus, observed_stimuli = hmm.forward(
    num_nodes,
    time_steps,
    initial_c,
)

### Learning with everything observed

In [ ]:
# This is necessary for mask computation.
observed_processing_modes: IntArray = np.array(observed_processing_modes)

In [ ]:
(
    lambda_0_hat,
    lambda_1_hat,
    learned_alpha,
    learned_beta,
    learned_gamma
) = learned_parameters(
    observed_processing_modes,
    observed_focus,
    observed_stimuli
)

Learned parameters ...

In [ ]:
learned_rates = [lambda_0_hat, lambda_1_hat]
learned_transition_matrix = np.array(
    [[1 - learned_gamma, 0, learned_gamma],
     [0, 1 - learned_gamma, learned_gamma],
     [learned_beta / 2, learned_beta / 2, 1 - learned_beta]]
)

In [ ]:
learned_hmm = HMM(
    transition=learned_transition_matrix,
    alpha=learned_alpha,
    sample_stimuli=hmm.sample_stimuli,
    states=hmm.states,
    rates=learned_rates
)

### Testing the learned model (everything observed)

In [ ]:
true_processing_modes, true_focus, observations = hmm.forward(
    num_nodes,
    time_steps,
    initial_c,
)

In [ ]:
original_joint_prob = hmm.infer(observations)
learned_joint_prob = learned_hmm.infer(observations)

In [ ]:
marginal_prob_C = np.sum(learned_joint_prob, axis=2)

estimated_C = np.argmax(marginal_prob_C, axis=1)
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = learned_hmm.sample_hidden_z(num_nodes, c)

In [ ]:
correct_C = np.sum(np.equal(estimated_C, true_processing_modes[:-1])) / (time_steps - 1)
correct_Z = np.sum(estimated_Z == true_focus) / ((time_steps - 1) * num_nodes)

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

Proportion of correct C estimations: 0.69
Proportion of correct Z estimations: 0.76


## Learning just from $\textbf{X}$ (full learning)

Compute $\hat{Z}_{t,i} = \argmax_z P(Z_{t,i} = z | \textbf{X} = \textbf{x})$ and $\hat{C}_t = \argmax_z P(C_t = z | \textbf{X} = \textbf{x})$

In [ ]:
# Whatever. We're just using some joint-prob, taking from above. :)
z_hat, c_hat = expectation_maximisation_hard_assignment(
    original_joint_prob, num_nodes=num_nodes
)

In [ ]:
z_hat.shape

(99, 8)

Learning ...

In [ ]:
epochs: int = 10 # lol.

In [ ]:
hmm = HMM(transition_matrix, alpha, lambda z: sample_poisson_stimuli(z, rates), states=[0, 1, 2], rates=rates)

In [ ]:
for _ in range(epochs):
    joint_prob = hmm.infer(observations)
    z_hat, c_hat = expectation_maximisation_hard_assignment(joint_prob, num_nodes=num_nodes)

    (
        lambda_0_hat,
        lambda_1_hat,
        learned_alpha,
        learned_beta,
        learned_gamma
    ) = learned_parameters(
        c_hat,
        z_hat,
        observations[:-1]
    )

    learned_rates = [lambda_0_hat, lambda_1_hat]
    learned_transition_matrix = np.array(
        [[1 - learned_gamma, 0, learned_gamma],
        [0, 1 - learned_gamma, learned_gamma],
        [learned_beta / 2, learned_beta / 2, 1 - learned_beta]]
    )

    hmm = HMM(learned_transition_matrix, alpha=learned_alpha, sample_stimuli=hmm.sample_stimuli, states=hmm.states, rates=learned_rates)


In [ ]:
learned_joint_prob = hmm.infer(observations)

In [ ]:
marginal_prob_C = np.sum(learned_joint_prob, axis=2)

estimated_C = np.argmax(marginal_prob_C, axis=1)
estimated_Z = np.zeros((time_steps, num_nodes), dtype=int)

for t, c in enumerate(estimated_C):
    estimated_Z[t] = hmm.sample_hidden_z(num_nodes, c)

correct_C = np.sum(np.equal(estimated_C, true_processing_modes[:-1])) / (time_steps - 1)
correct_Z = np.sum(estimated_Z == true_focus) / ((time_steps - 1) * num_nodes)

print(f"Proportion of correct C estimations: {correct_C:.2f}")
print(f"Proportion of correct Z estimations: {correct_Z:.2f}")

Proportion of correct C estimations: 0.57
Proportion of correct Z estimations: 0.37
